# aspects


Aspects provide additional functionality to the classic Object-Oriented pattern.

https://en.wikipedia.org/wiki/Aspect-oriented_programming



## introduction


Types of aspects:

- affect - run after the code
- prefect - run before the code
- inflect - make available part of the affection within the code
- reflect - describe the current affection
- infect - replace part of code with result of another
- effect - return a list of affections the occured or will occur if reference is provided
- expect - deriving from the context find the variables involved and if they do not meet expectation create a "soft assertion"?
- inspect - show debugging information surrounding the specified code


TODO: aspect babel plugin. This will use the transpiler.

TODO: depends on transpiling kernels


## transpiler bindings

Transpile calls that bind functionality to the neessary directives.



### inspect

TODO: inspect, find calls to aspect and interpret them up front.


#### the code

inspect every statement?


In [ ]:
var importer = require('../Core')
var transpile = importer.import('transpile code')

function inspect(ctx, line, filename) {
    console.log(JSON.stringify(ctx), line, filename)
}

function inspectTemplate(ctx, line, filename) {
    // code inserted in to transpiled module
    inspect(ctx, line, filename)
}

function transpileInspect(code) {
    return transpile([
        `//FunctionDeclaration`,
        (ctx) => {
            // TODO: 'replace' transpiler
            var inspect = selectAst(`//FunctionDeclaration`, inspectTemplate)
            var ctx = selectDom(`.//*[./Identifier/@name="ctx"]`, inspect)
            console.log(ctx)
            //ctx.replaceWith(selectDom())
            var parent = selectDom(`parent::*`, ctx)
            parent.insertBefore(inspect, ctx)
        }
    ], code)
}

module.exports = {
    inspect,
    inspectTemplate,
    transpileInspect,
}


#### test statement inspector



In [1]:
var importer = require('../Core')
var transpileInspect = importer.import('inspect every statement')

var code = `
var escodegen = require('escodegen');
var importer = require('../Core');

addImports('some code')

`

if(typeof $$ != 'undefined') {
    $$.mime({'text/plain': transpileInspect(code)})
    
    /*
    expected output 
var addImports = importer.import('add missing imports')
var escodegen = require('escodegen');
*/
    
}


compiling /Users/briancullinan/jupyter_ops/Core/files.ipynb[2] aka ,### list project files
compiling /Users/briancullinan/jupyter_ops/Core/files.ipynb[3] aka common ignore paths
compiling /Users/briancullinan/jupyter_ops/Core/aspects.ipynb[0] aka aspect oriented
compiling /Users/briancullinan/jupyter_ops/Core/languages.ipynb[2] aka transpile code
compiling /Users/briancullinan/jupyter_ops/Core/syntax.ipynb[13] aka html to tree
compiling /Users/briancullinan/jupyter_ops/Core/syntax.ipynb[16] aka select ast
compiling /Users/briancullinan/jupyter_ops/Core/syntax.ipynb[11] aka tree to html
compiling /Users/briancullinan/jupyter_ops/Core/patterns.ipynb[2] aka ### select tree
compiling /Users/briancullinan/jupyter_ops/Core/patterns.ipynb[0] aka ,### walk tree


TypeError: transpileInspect is not a function